# Downsample videos to target fps

We downsampled our videos to 5fps to ease the memory constraints.

Replace the DATA path with your video path of the original .avi files.

In [ ]:
import cv2
from tqdm import tqdm

In [ ]:
def downsample_avi_to_mp4(input_avi, output_mp4, target_fps=1):
    """Downsample an AVI video to MP4 with a target FPS."""
    cap = cv2.VideoCapture(input_avi)
    if not cap.isOpened():
        raise IOError(f"Cannot open {input_avi}")

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    if original_fps % target_fps != 0:
        raise ValueError("Original FPS must be divisible by target FPS for integer downsampling.")
    stride = int(round(original_fps / target_fps))
    if stride < 1:
        raise ValueError(f"Target FPS {target_fps} is too high for source FPS {original_fps}")

    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Good cross-platform choice
    out = cv2.VideoWriter(output_mp4, fourcc, target_fps, (width, height))
    if not out.isOpened():
        raise IOError(f"Cannot write to {output_mp4}")

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % stride == 0:
            out.write(frame)

        frame_idx += 1

    cap.release()
    out.release()
    print(f"Downsampled video saved to {output_mp4} at {target_fps} FPS")


In [ ]:
DATA = "path/to/your/videos" # Replace with your path

In [ ]:
target_fps = 5
# iterate over all people
for subject in tqdm(range(1,21)):
    #iterate over 1st, 2nd and 3rd run for cam2
    for run in range(1,4):
        RUN = f'{subject:02}_run{run}_cam_2'
        input_path = f'{DATA}/Vacuum_pump/{subject:02}/Full_recordings/{RUN}.avi'
        output_path = f'{DATA}/Vacuum_pump/{subject:02}/Full_recordings/{RUN}_{target_fps}fps.mp4'
        downsample_avi_to_mp4(input_path, output_path, target_fps=target_fps)